In [1]:
import numpy as np
import time
import stock
import kline

In [2]:
kline.Plote('SZ002230').show()

Box(children=(Button(description='上一页', style=ButtonStyle()), Button(description='下一页', style=ButtonStyle()), …

Output()

In [1]:
kline.Plote('SH600519').show()

NameError: name 'kline' is not defined

In [5]:
kline.Plote('SH:601099').show()

Box(children=(Button(description='上一页', style=ButtonStyle()), Button(description='下一页', style=ButtonStyle()), …

Output()